In [1]:
import time
from selenium.webdriver import Firefox
import pandas as pd

In [2]:
driver = Firefox()

In [ ]:
funds = [
    (
        "https://www.fullertonfund.com/fund/fullerton-sgd-cash-fund-a/",
        "data/Fullerton SGD Cash Fund A",
    ),
    (
        "https://www.fullertonfund.com/fund/fullerton-sgd-liquidity-fund-a",
        "Fullerton SGD Liquidity Fund A",
    ),
]

In [ ]:
for fund_url, file_name in funds:
    driver.get(fund_url)

    time.sleep(0.5)

    if driver.find_elements(by="class name", value="button-close"):
        driver.find_element(by="class name", value="button-close").click()

    n_pages = int(
        driver.find_element("id", "fund-range-history_paginate")
        .find_elements("class name", "paginate_button")[-2]
        .text
    )

    rows = []

    for i in range(n_pages - 1):
        rows.extend(
            driver.find_element("id", "fund-range-history").text.split("\n")[1:]
        )
        driver.find_element("id", "fund-range-history_next").click()

    rows.extend(driver.find_element("id", "fund-range-history").text.split("\n")[1:])

    df = pd.DataFrame(
        pd.Series(rows[::-1]).str.split().apply(lambda x: x[:2]).to_list()
    ).set_axis(["date", "NAV"], axis=1)
    df["date"] = pd.to_datetime(df["date"], dayfirst=True)
    df["NAV"] = df["NAV"].str.replace("$", "").astype(float)
    df.set_index("date").to_csv(f"data/{file_name}.csv")

driver.quit()